In [50]:
import pandas as pd
import numpy as np
import math
import random


#ignore_features = ['58', '16', '7', '25', '58', '23', '20']
ignore_features = []
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [51]:
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [52]:
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [53]:
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error
    


In [54]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [57]:
testlabelnum = [value[0] for value in testlabel]
labnum = [value[0] for value in testlabel]


In [55]:
ylogregtest = logreg.fit(train, lab).predict(test)


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
print eval(ylogregtest, testlabelnum)

0.889583128018


In [59]:
ylogregquiz = logreg.fit(train, lab).predict(quiz)

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [65]:
ylogregquiz = logreg.fit(train, lab).predict(quiz)
csvfile = "logreg2.csv"

with open(csvfile,"w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in ylogregquiz:
        writer.writerow([val])
        

In [63]:
len(ylogregquiz)

31709

In [66]:
from sklearn.svm import SVC
svc = SVC()

ysvctest = svc.fit(train, lab).predict(test)
print eval(ysvctest, testlabelnum)

0.77737262245


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [68]:
ysvcquiz = svc.fit(train, lab).predict(quiz)
csvfile = "svc.csv"

with open(csvfile,"w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in ysvcquiz:
        writer.writerow([val])

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [72]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=600, learning_rate=1)

yadaboosttest = adaboost.fit(train, lab).predict(test)
print eval(yadaboosttest, testlabelnum)

0.900335074406


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

ysgdtest = sgd.fit(train, lab).predict(test)
print eval(ysgdtest, testlabelnum)

0.837163693703


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
